In [ ]:
#%%capture
!pip install --no-cache-dir descartes
!pip install --no-cache-dir shapely
!pip install setuptools
!pip install ez_setup
!pip install datascience
#!pip download GDAL
#!pip install greenwich

In [ ]:
%matplotlib inline
import os
from datetime import datetime
from descartes import PolygonPatch
import matplotlib as mpl
from matplotlib.collections import PatchCollection
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from datascience import *
import espm_module #MAKE SURE THAT ESPM_MODULE.PY IS IN THE SAME DIRECTORY AS THIS NOTEBOOK
from shapely import geometry as sg, wkt
from espm_module import CalAdaptRequest, GBIFRequest
plt.style.use('seaborn')

# ESPM / IB 105

The goal of this notebook is to access and integrate diverse data sets to visualize correlations and discover patterns to address questions of species’ responses to environmental change. We will use programmatic tools to show how to use Berkeley resources such as the biodiversity data from biocollections and online databases, field stations, climate models, and other environmental data.

Before we begin analyzing and visualizing biodiversity data, this introductory notebook will help familiarize you with the basics of programming in Python. This notebook will cover fundamental Python syntax, Pandas dataframes, and visualization using Matplotlib.

## Table of Contents


    
1 - [Python Basics](#python basics)

2 - [Pandas Basics](#pandas basics)

3 - [Matplotlib Basics](#matplotlib)

4 - [GBIF API](#gbif)

5 - [Cal-Adapt API](#adapt)

### Part 1: Python basics <a id='python basics'></a>
Before getting into the more advanced analysis techniques that will be required in this course, we need to cover a few of the foundational elements of programming in Python.
#### A. Expressions
The departure point for all programming is the concept of the __expression__. An expression is a combination of variables, operators, and other Python elements that the language interprets and acts upon. Expressions act as a set of instructions to be fed through the interpreter, with the goal of generating specific outcomes. See below for some examples of basic expressions.

In [ ]:
# Examples of expressions:

2 + 2

'me' + ' and I'

12 ** 2

6 + 4

You will notice that only the last line in a cell gets printed out. If you want to see the values of previous expressions, you need to call `print` on that expression. Try adding `print` statements to some of the above expressions to get them to display.

#### B. Variables
In the example below, `a` and `b` are Python objects known as __variables__. We are giving an object (in this case, an `integer` and a `float`, two Python data types) a name that we can store for later use. To use that value, we can simply type the name that we stored the value as. Variables are stored within the notebook's environment, meaning stored variable values carry over from cell to cell.

In [ ]:
a = 4
b = 10/5

Notice that when you create a variable, unlike what you previously saw with the expressions, it does not print anything out.

In [ ]:
# Notice that 'a' retains its value.
print(a)
a + b

#### C. Lists
The next topic is particularly useful in the kind of data manipulation that you will see throughout 101B. The following few cells will introduce the concept of __lists__ (and their counterpart, `numpy arrays`). Read through the following cell to understand the basic structure of a list. 

A list is an ordered collection of objects. They allow us to store and access groups of variables and other objects for easy access and analysis. Check out this [documentation](https://www.tutorialspoint.com/python/python_lists.htm) for an in-depth look at the capabilities of lists.

To initialize a list, you use brackets. Putting objects separated by commas in between the brackets will add them to the list. 

In [ ]:
# an empty list
lst = []
print(lst)

# reassigning our empty list to a new list
lst = [1, 3, 6, 'lists', 'are' 'fun', 4]
print(lst)

To access a value in the list, put the index of the item you wish to access in brackets following the variable that stores the list. Lists in Python are zero-indexed, so the indicies for `lst` are 0, 1, 2, 3, 4, 5, and 6.

In [ ]:
# Elements are selected like this:
example = lst[2]

# The above line selects the 3rd element of lst (list indices are 0-offset) and sets it to a variable named example.
print(example)

#### D. Functions

So far, you've learnt how to assign variables to certain values. Now, let's learn how we can manipulate some of these values.

Let's say we want to perform a certain operation on many different inputs that will produce distinct outputs. What do we do? We write a **_function_**.

A function is a block of code which works a lot like a machine: it takes an input, does something to it, and produces an output.

The input is put between brackets and can also be called the _argument_ or _parameter_. Functions can have multiple arguments.

Try running the cell below after changing the variable _name_:

In [ ]:
# Edit this cell to your own name!
name = "John Doe"

# Our function
def hello(name):
    return "Hello " + name + "!"

hello(name)

Interesting, right? Now, you don't need to write 10 different lines with 10 different names to print a special greeting for each person. All you need to is write one function that does all the work for you!

Now, let's write our own function. Let's look at the following rules:

**Defining**

* All functions must start with the "def" keyword.
* All functions must have a name, followed by parentheses, followed by a colon. Eg. def hello( ):
* The parentheses may have a variable that stores its arguments (inputs)
* All functions must have a "return" statement which will return the output. Think of a function like a machine. When you put something inside, you want it to return something. Hence, this is very important.

**Calling**

After you define a function, it's time to use it. This is known as _calling_ a function.

To call a function, simply write the name of the function with your input variable in brackets (argument).

In [ ]:
# Complete this function
def my_first_function(argument):
    return # function must return a value - REPLACE THIS LINE
           # have this function return the input

name = ... # FILL IN YOUR NAME HERE, in quotes

# Calling our function below...
my_first_function(name)

Great! Now let's do some math. Let's write a function that returns the square of the input.

Try writing it from scratch!

In [ ]:
# square function 
def square(num):
    return ...   # YOUR CODE HERE

# this should output 25!
# try inputting different numbers instead of 5 and see what happens
square(5)

Neat stuff! Try different inputs and check if you get the correct answer each time.

You've successfully written your first function from scratch!

### Part 2: Pandas Dataframes <a id='pandas basics'></a>

We will be using Pandas dataframes for several portions of this notebook to organize and analyze our data. [Pandas](http://pandas.pydata.org/pandas-docs/stable/) is one of the most widely used Python libraries in data science. It is commonly used for data cleaning, and with good reason: it’s very powerful and flexible, among many other things. 

#### Creating dataframes

The rows and columns of a pandas dataframe are essentially a collection of lists stacked on top/next to each other. For example, if I wanted to store the top 10 movies and their ratings in a datatable, I could create 10 lists that each contain a rating and a corresponding title, and these lists would be the rows of the table:

In [ ]:
top_10_movies = pd.DataFrame(data=np.array(
            [[9.2, 'The Shawshank Redemption (1994)'],
            [9.2, 'The Godfather (1972)'],
            [9., 'The Godfather: Part II (1974)'],
            [8.9, 'Pulp Fiction (1994)'],
            [8.9, "Schindler's List (1993)"],
            [8.9, 'The Lord of the Rings: The Return of the King (2003)'],
            [8.9, '12 Angry Men (1957)'],
            [8.9, 'The Dark Knight (2008)'],
            [8.9, 'Il buono, il brutto, il cattivo (1966)'],
            [8.8, 'The Lord of the Rings: The Fellowship of the Ring (2001)']]), columns=["Rating", "Movie"])
top_10_movies

Alternatively, we can store data in a dictionary instead of in lists. A dictionary keeps a mapping of keys to a set of values, and each key is unique. Using our top 10 movies example, we could create a dictionary that contains ratings a key, and movie titles as another key.

In [ ]:
top_10_movies_dict = {"Rating" : [9.2, 9.2, 9., 8.9, 8.9, 8.9, 8.9, 8.9, 8.9, 8.8], 
                      "Movie" : ['The Shawshank Redemption (1994)',
                                'The Godfather (1972)',
                                'The Godfather: Part II (1974)',
                                'Pulp Fiction (1994)',
                                "Schindler's List (1993)",
                                'The Lord of the Rings: The Return of the King (2003)',
                                '12 Angry Men (1957)',
                                'The Dark Knight (2008)',
                                'Il buono, il brutto, il cattivo (1966)',
                                'The Lord of the Rings: The Fellowship of the Ring (2001)']}

Now, we can use this dictionary to create a table with columns `Rating` and `Movie`

In [ ]:
top_10_movies_2 = pd.DataFrame(data=top_10_movies_dict, columns=["Rating", "Movie"])
top_10_movies_2

Notice how both ways return the same table! However, the list method created the table by essentially taking the lists and making up the rows of the table, while the dictionary method took the keys from the dictionary to make up the columns of the table. In this way, dataframes can be viewed as a collection of basic data structures, either through collecting rows or columns. 

#### Reading in Dataframes

Generally, when performing data analysis, most datatables are premade/pulled from another source and created in a form that is easily read into a pandas method, which creates the table for you. A common file type that is used for data is a Comma-Separated Values (.csv) file, which stores tabular data. It is not necessary for you to know exactly how .csv files store data, but you should know how to read a file in as a pandas dataframe. 

We will read in a .csv file that contains data from Spring 2017 about economic time value and closure.

In [ ]:
# Run this cell to read in the table
vot = pd.read_csv("vots.csv")

The `pd.read_csv` function expects a path to a .csv file as its input, and will return a data table created from the data contained in the csv.
We have provided `vots.csv` in the data directory, which is all contained in the current working directory (aka the folder this assignment is contained in). For this reason, we must specify to the `read_csv` function that it should look for the csv in the data directory, and the `/` indicates that `vots.csv` can be found there. 

Here is a sample of some of the rows in this datatable:

In [ ]:
vot.head()

#### Indexing Dataframes

Oftentimes, tables will contain a lot of extraneous data that muddles our data tables, making it more difficult to quickly and accurately obtain the data we need. To correct for this, we can select out columns or rows that we need by indexing our dataframes. 

The easiest way to index into a table is with square bracket notation. Suppose you wanted to obtain all of the closure data from the data. Using a single pair of square brackets, you could index the table for `"closure"`

In [ ]:
# Run this cell and see what it outputs
vot["closure"]

Notice how the above cell returns an array of all the closure values in their original order.
Now, if you wanted to get the first closure value from this array, you could index it with another pair of square brackets:

In [ ]:
vot["closure"][0]

Pandas columns have many of the same properties as numpy arrays. Keep in mind that pandas dataframes, as well as many other data structures, are zero-indexed, meaning indexes start at 0 and end at the number of elements minus one. 

If you wanted to create a new datatable with select columns from the original table, you can index with double brackets.

In [ ]:
## Note: .head() returns the first five rows of the table
vot[['pclo', 'tclo', 'kclo', 'pvot', 'tvot', 'kvot']].head()

Alternatively, you can also get rid of columns you dont need using `.drop()`

In [ ]:
vot.drop("gender", axis=1).head()

Finally, you can use square bracket notation to index rows by their indices with a single set of brackets. You must specify a range of values for which you want to index. For example, if I wanted the 20th to 30th rows of `accounts`:

In [ ]:
vot[20:31]

#### Filtering Data

As you can tell from the previous, indexing rows based on indices is only useful when you know the specific set of rows that you need, and you can only really get a range of entries. Working with data often involves huge datasets, making it inefficient and sometimes impossible to know exactly what indices to be looking at. On top of that, most data analysis concerns itself with looking for patterns or specific conditions in the data, which is impossible to look for with simple index based sorting.   

Thankfully, you can also use square bracket notation to filter out data based on a condition. Suppose we only wanted closure and vot data for individuals taller than 160 centimeters:

In [ ]:
vot[vot["height"] >= 170][["closure", "vot"]]

The `vot` table is being indexed by the condition `vot["height"] >= 170`, which returns a table where only rows that have a "height" greater than $170$ is returned. We then index this table with the double bracket notation from the previous section to only get the closure and vot columns.

Suppose now we wanted a table with data from English speakers, and where the t VOT was less than .01 or k VOT is greater than .08.

In [ ]:
vot[(vot["language"] == "English") & ((vot["tvot"] < .01) | (vot["kvot"] > .08))]

Many different conditions can be included to filter, and you can use `&` and `|` operators to connect them together. Make sure to include parantheses for each condition!

Another way to reorganize data to make it more convenient is to sort the data by the values in a specific column. For example, if we wanted to find the the tallest person in our data set, we could sort the table by height:

In [ ]:
vot.sort_values("height")

But wait! The table looks like it's sorted in increasing order. This is because `sort_values` defaults to ordering the column in ascending order. To correct this, add in the extra optional parameter

In [ ]:
vot.sort_values("height", ascending=False)

Now we can clearly see that the tallest person was 185.42 cm tall.

#### Useful Functions for Numeric Data

Here are a few useful functions when dealing with numeric data columns.
To find the minimum value in a column, call `min()` on a column of the table.

In [ ]:
vot["pvot"].min()

To find the maximum value, call `max()`.

In [ ]:
vot["pvot"].max()

And to find the average value of a column, use `mean()`.

In [ ]:
vot["pvot"].mean()

### Part 3: Visualization <a id='matplotlib'></a>

Now that you can read in data and manipulate it, you are now ready to learn about how to visualize data. Matplotlib is a very popular Python library that is used to visualize data, and it is the one that we will be discussing below.

One of the advantages of pandas is its built-in plotting methods. We can simply call `.plot()` on a dataframe to plot columns against one another. All that we have to do is specify which column to plot on which axis. We include the extra variable `kind='scatter'` to override the default lineplot.

In [ ]:
vot.plot(x='pvot', y='kvot', kind='scatter')

The base package for most plotting in Python is `matplotlib`. Below we will look at how to plot with it. First we will extract the columns that we are interested in, then plot them in a scatter plot. Note that `plt` is the common convention for `matplotlib.pyplot`.

In [ ]:
closure = vot['tclo']
height = vot['height']

#Plot the data by inputting the x and y axis
plt.scatter(closure, height)

# we can then go on to customize the plot with labels
plt.xlabel("T Closure")
plt.ylabel("Height")

Though matplotlib is sometimes considered an "ugly" plotting tool, it is powerful. It is highly customizable and is the foundation for most Python plotting libraries. Check out the [documentation](https://matplotlib.org/api/pyplot_summary.html) to get a sense of all of the things you can do with it, which extend far beyond scatter and line plots. 

#### Question 5: Plotting

Try plotting two of the closure columns against one another.

In [ ]:
#pclo = ...
#kclo = ...

#plt.scatter(pclo, kclo)
#plt.xlabel(...)
#plt.ylabel(...)

# note: plt.show() is the equivalent of print, but for graphs
#plt.show()

Over the past few sections, we provided you with a brief introduction to Python and relevant Python libraries! The above tutorial is by no means comprehensive, so if you desire more information on a specific topic, consult its documentation! Now let's dive into some data analysis.

---

# GBIF API<a id='gbif'></a>

The Global Biodiversity Information Facility has created an API that we can use to get data about different species at the [GBIF Web API](http://www.gbif.org/developer/summary). It utilizes pagination to split up the returned JSON records, so we need to request each successive page to gather all the results. Then, we can request the available occurrence records for a particular species and plot their locations. 

When performing data analysis, it is always important to define a question that you seek the answer to. The goal of finding the answer to this question will ultimately drive the queries and analysis styles you choose to use/write.

For this example, we are going to ask: where have Argia agrioides California Dancer dragonfly been documented? Are there records at any of our field stations?

Here we're going to make a request for the species whose scientific name is 'Argia agrioides', also known as the [California dancer](https://www.google.com/search?q=Argia+agrioides&rlz=1C1CHBF_enUS734US734&source=lnms&tbm=isch&sa=X&ved=0ahUKEwji9t29kNTWAhVBymMKHWJ-ANcQ_AUICygC&biw=1536&bih=694). We're going to plot their locations and display other data results. Keep in mind that this query/search can be done for ANY taxon. All you need to do is change the input parameters!

In [ ]:
req = GBIFRequest()  #creating a request to the API
params = {'scientificName': 'Argia agrioides'}  #setting our parameters (the specific species we want)
pages = req.get_pages(params)  #using those parameters to complete the request
# Filter all the returned records for valid locations.
records = [rec for page in pages for rec in page['results']
           if rec.get('decimalLatitude')]
# Make a list of coordinate pairs for plotting.
coords = [(r['decimalLongitude'], r['decimalLatitude']) for r in records]
xs, ys = zip(*coords)

The next thing we are going to do is filter through our records to get a count of specimens as well as their respective collections. 

In [ ]:
specimencounter = 0    #create a counter for the number of specimens (vs number of observations)
recordholder = np.array([])   #create arrays that we will hold information in for table (record # and collection)
collectionholder = np.array([])
for i in range(0, len(records)):   #iterates through all records
    if records[i]['collectionCode'] != 'Observations': #checks if the collection code is not observations
        specimencounter += 1
        recordholder= np.append(recordholder, i) #adding record number and collection to arrays
        collectionholder = np.append(collectionholder, records[i]['collectionCode'])
        
collectionstable = Table().with_columns("Record", recordholder, "Collection", collectionholder) #creating table

print("Total number of records: " + str(len(records)))
print("Number of specimens: " + str(specimencounter))
print("Number of observations: " + str((len(records) - specimencounter)))

collectionstable.show(len(collectionholder)) #change the parameter in len to alter size of table/omitted rows

We can then plot the species:

In [ ]:
# Make a plot of species' locations below.
plt.plot(xs, ys, 'ro', alpha=0.5)

For all the reserves, we will need to send a request to GeoJSON, which is a format for encoding a variety of geographic data structures. With this code, we can request GeoJSON for all reserves and plot ocurrences of the species.

In [ ]:
# This is the url where are data is located 
url = 'https://ecoengine.berkeley.edu/api/layers/reserves/features/'
# Make a request, which receives data from this url. Then iterate throuh each feature so we can collect data about them.
reserves = requests.get(url, params={'page_size': 30}).json()
geoms = {feat['properties']['name']: sg.asShape(feat['geometry']) for feat in reserves['features']}

In [ ]:
# Buffer the reserves so we can view them at a regional scale.
patches = [PolygonPatch(geom.buffer(0.25)) for geom in geoms.values()]
fig, ax = plt.subplots()
colors = 100 * np.random.rand(len(patches))
p = PatchCollection(patches, cmap='Accent', alpha=0.4)
p.set_array(np.array(colors))
ax.add_collection(p)
ax.autoscale()

Let's try and zoom in to the Santa Cruz Island Reserve where we have a single occurrence, and plot it similarly. 

In [ ]:
# Collect data about what we want to graph.
fig, ax = plt.subplots()
g = geoms['Santa Cruz Island Reserve']
poly = PolygonPatch(g)
ax.add_patch(poly)
# Plot a graph representing the Santa Cruz Island Reserve using the data we've accumulated in these variables.
ax.plot(xs, ys, 'ro', clip_path=poly)
xmin, ymin, xmax, ymax = g.bounds
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

In the same way, we can parse the WKT geometry for Blodgett, Hopland, and Sagehen stations and plot them individually.

In [ ]:
station_urls = {
    'Blodgett': 'https://raw.githubusercontent.com/BNHM/spatial-layers/master/wkt/BlodgettForestResearchStation.wkt',
    'Hopland': 'https://raw.githubusercontent.com/BNHM/spatial-layers/master/wkt/HoplandREC.wkt',
    'Sagehen': 'https://raw.githubusercontent.com/BNHM/spatial-layers/master/wkt/SagehenCreekFieldStation.wkt'
}
stn_features = [{'id': name, 'geometry': wkt.loads(requests.get(url).text)}
                for name, url in station_urls.items()]
# Reduce geometric complexity, we need smaller geometry serializations to circumvent 
# URL length restrictions.
tol = .0001
for feat in stn_features:
    feat['geometry'] = feat['geometry'].buffer(tol).simplify(tol)

In [ ]:
f, axes = plt.subplots(1, len(stn_features))
f.set_size_inches((12, 6))

# Title our graphs.
for ax, feat in zip(axes, stn_features):
    ax.add_patch(PolygonPatch(feat['geometry']))
    ax.set_title(feat['id'])
    ax.autoscale()

This will use the [Cal-Adapt](http://api.cal-adapt.org/api/) web API to access a remote raster data source and preview a rendered image. In this case, the data source contains images of locations and temperature information corresponding to each one. 

In [ ]:
url = 'http://api.cal-adapt.org/api/series/tasmax_year_CanESM2_rcp85/rasters/'
json = requests.get(url).json()
tifurl = json['results'][0]['image']
title = json['results'][0]['slug']

In [ ]:
import greenwich

with greenwich.open(tifurl) as r:
    arr = r.masked_array()

Now we need to visualize the resulting masked array. The array values are maximum temperature in Kelvin.

In [ ]:
# Use Matplotlib to create a color-coded plot of the resulting masked array:
plt.imshow(arr, cmap='spectral')
plt.title(title)
plt.colorbar()

In [ ]:
# Use Matplotlib to plot a histogram (arr should be un-masked here)

h = plt.hist(arr.compressed(), 20)

---

# Cal-Adapt API<a id='adapt'></a>

As we look closer at the Cal-Adapt API, the following will ease working with time series raster data. It will request an entire time series for any geometry and return a Pandas `DataFrame` object. This will make displaying the data much easier with Pandas' built in methods. 

In [ ]:
# Creating Pandas dataframe by combining requested time series with records_g
req = CalAdaptRequest()
records_g = [dict(rec, geometry=sg.Point(rec['decimalLongitude'], rec['decimalLatitude']))
             for rec in records]
df = req.concat_features(records_g, 'gbifID')

In [ ]:
df.iloc[:,:9].plot()
plt.title('Argia agrioides - %s' % req.slug)

We can map the projected means for occurrence locations.

In [ ]:
tmax_means = df.mean(axis=0)
# Filter GBIF records where we have climate data.
records_rvals = [rec for rec in records_g if rec['gbifID'] in tmax_means.index]
coords = [(r['decimalLongitude'], r['decimalLatitude']) for r in records_rvals]
xs2, ys2 = zip(*coords)
#plt.scatter(xs2, ys2, c=tmax_means, cmap='viridis', alpha=0.5)
norm = mpl.colors.Normalize()
size = np.pi * (15 * norm(tmax_means)) ** 2 
plt.scatter(xs2, ys2, c=tmax_means, s=size, cmap='viridis', alpha=0.5)
plt.colorbar()

In [ ]:
#plt.figure(figsize=(12, 6))
df.iloc[:,:7].plot.box()
plt.title('Argia agrioides - %s' % req.slug)

Taking a look at minimum temperatures now, here are the temperature distributions by decade.

In [ ]:
dfs = []
names = ('tasmin_year_CanESM2_rcp45', 'tasmin_year_CanESM2_rcp85')
for name in names:
    req = CalAdaptRequest(name)
    dfs.append(req.concat_features(records_g, 'gbifID'))

In [ ]:
for name, df in zip(names, dfs):
    t = df.resample('10A').mean().transpose()
    t.columns = t.columns.year
    t.plot.box()
    plt.title(name)

Make plots for historical and projected temperatures at three station locations. Once again, Pandas gives us a lot of functionality in displaying our data beautifully. 

In [ ]:
dfstns_lst = []
names = ('tasmax_year_CanESM2_historical', 'tasmax_year_CanESM2_rcp85')
# Iterate through each name and make requests using the CalAdapt API. We can also use Pandas to display this data efficiently.
for name in names:
    req = CalAdaptRequest(name)
    d = req.concat_features(stn_features)
    dfstns_lst.append(d)
    #d.plot()
    #plt.title(name)
dfstns = pd.concat(dfstns_lst)

In [ ]:
dfstns.describe()

In [ ]:
dfstns.plot()

In [ ]:
# Decadal average and difference from 20 years prior.
dfstns.resample('10A').mean().diff(periods=2).dropna()

In [ ]:
dfstns.resample('10A').mean().diff().plot()

Comparing California Oak species:
#1 Search on Quercus douglassi (Blue Oak), Quercus lobata (Valley Oak), Quercus durata (Leather Oak), Q. agrifolia (Coast Live Oak) within Bounding box of the field stations. 

Which field stations have which species? Rerun charting of localities on observed climate to answer: do they all experience the same yearly average maximums and minimums? Which species has the highest max tolerance? Minimum? How does that compare wtih the future climate models for those localities? Which species is predicted to experience the most change in the future?

#2 What are the specimens that occurs in Sagehen Creek FS (or any one of the three field stations)? (SCFS most likely to have checklist in Ecoengine...) 

Use: GBIF API, ecoengine api for checklists (also bigcb.berkeley.edu, although may not be available as web service???)
Search GBIF for specimens within the SCFS polygon. Create a unique species list.
Compare with the unique documented species list with the checklist species. Which species are expected but not documented? Are there any documented but not on the checklist?

What other meaningful spatial data can we intersect with species occurrence? With the field station polygons in order to compare how our field stations sample different habitats and ecoregions?